In [1]:
!pip install pymc
!pip install numpyro
!pip install aeppl
!pip install jax

In [2]:
'''import matplotlib.pyplot as plt

import pyGMs as gm
import numpy as np
import torch

import pyro
import numpyro
import numpyro.infer
import numpyro.optim
import numpyro.distributions as dist
import pyro.poutine as poutine
from jax import random
import jax.numpy as jnp

import pandas as pd

import requests                                      # reading data
from io import StringIO
import re                                            # regular expressions

# Fix the random seed for reproducibility
# !! Important !! : do not change this
seed = 123
pyro.set_rng_seed(seed)
#np.random.seed(seed);
#torch.manual_seed(seed);'''

'import matplotlib.pyplot as plt\n\nimport pyGMs as gm\nimport numpy as np\nimport torch\n\nimport pyro\nimport numpyro\nimport numpyro.infer\nimport numpyro.optim\nimport numpyro.distributions as dist\nimport pyro.poutine as poutine\nfrom jax import random\nimport jax.numpy as jnp\n\nimport pandas as pd\n\nimport requests                                      # reading data\nfrom io import StringIO\nimport re                                            # regular expressions\n\n# Fix the random seed for reproducibility\n# !! Important !! : do not change this\nseed = 123\npyro.set_rng_seed(seed)\n#np.random.seed(seed);\n#torch.manual_seed(seed);'

In [3]:

import pandas as pd

#getting info
df = pd.read_csv('games.csv')


#print(df['white_id'])
white_ids = df['white_id']
black_ids =  df["black_id"]
winners = df["winner"]

print(white_ids[:5])

0         bourgris
1             a-00
2           ischia
3    daniamurashov
4        nik221107
Name: white_id, dtype: object


In [4]:
import numpy as np

uniqueppl = white_ids.tolist()
uniqueppl.extend(black_ids.tolist())

uppl = np.array(list(set(uniqueppl)))#set of unique people

n_players = len(uppl)
uppl = []

# reformat gamesplayed -> winner first, loser second
gamesplayed = []
for game in range(len(df)):
    white_id = df.iloc[game].white_id
    black_id = df.iloc[game].black_id


    if df.iloc[game].victory_status == "mate":
      uppl.append(white_id)
      uppl.append(black_id)
      if df.iloc[game].winner == "white":
          gamesplayed.append([white_id, black_id])
      if df.iloc[game].winner == "black":
          gamesplayed.append([black_id, white_id])

    #if df.iloc[game].victory_status == "draw":
          #gamesplayed.append([black_id, white_id])
          #gamesplayed.append([white_id, black_id])



print(gamesplayed[0:5])
uppl = np.array(list(set(uppl)))
n_players = len(uppl)

[['ischia', 'a-00'], ['daniamurashov', 'adivanov2009'], ['nik221107', 'adivanov2009'], ['daniel_likes_chess', 'mirco25'], ['daniel_likes_chess', 'anaissac']]


In [5]:
import numpy as np
import pymc as pm

# convert str to int
player_to_id = {player: i for i, player in enumerate(uppl)}
player_ids = np.array([[player_to_id[winner], player_to_id[loser]] for winner, loser in gamesplayed])

# check sizes -> something is wrong here. tracce has more players than uppl
print(len(uppl))
print(len(player_to_id))
print(len(player_ids))

with pm.Model() as model:
    player_sd = pm.HalfNormal("player_sd", sigma=1.0)
    player_skills_raw = pm.Normal(
        "player_skills_raw", 0.0, sigma=1.0, shape=(n_players,)
    )
    player_skills = pm.Deterministic("player_skills", player_skills_raw * player_sd)

    winner_ids = player_ids[:, 0]
    loser_ids = player_ids[:, 1]

    logit_skills = player_skills[winner_ids] - player_skills[loser_ids]
    lik = pm.Bernoulli(
        "win_lik", logit_p=logit_skills, observed=np.ones(winner_ids.shape[0])
    )

    # Sample using the numpyro backend for JAX
    trace = pm.sample(4000, tune=1000, chains=4, cores=1, nuts_sampler='numpyro', random_seed=42)

6216
6216
6325


  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

In [6]:
import arviz as az

summary = az.summary(trace, kind="stats")

In [7]:

# filter out raw player skills (useless)
player_skills_summary = summary[~summary.index.str.startswith('player_skills_raw')]
player_skills_summary = player_skills_summary[1:]

player_means = player_skills_summary['mean']
player_sds = player_skills_summary['sd']
print(len(uppl))
print(len(player_means))
print(len(player_sds))

# Create df
player_skills_df = pd.DataFrame({
    'Player Name': uppl,
    'Skill Mean': player_means.values,
    'Skill SD': player_sds.values
})

sorted_player_skills_df = player_skills_df.sort_values(by='Skill Mean', ascending=False)
print(sorted_player_skills_df)

6216
6216
6216
             Player Name  Skill Mean  Skill SD
2620          brandonbos       4.893     1.496
727           fabian1104       4.528     1.545
1694           smilsydov       4.504     1.534
4192           chesscarl       4.264     1.566
1591              apalin       4.231     1.601
...                  ...         ...       ...
4199          sveenemand      -4.449     1.562
784   lucatiel-of-mirrah      -4.542     1.532
3472           jarrydlee      -4.546     1.539
1223               burks      -4.709     1.512
75                marigw      -5.817     1.364

[6216 rows x 3 columns]


In [8]:
def numgamess(player_id):
  wl = len(df.loc[df['white_id'] == player_id])
  bl = len(df.loc[df['black_id'] == player_id])
  return wl+bl

In [9]:
# Concatenate white_id and black_id columns to get all player ids
import math

onlymates = df.loc[df['victory_status'] == "mate"]

all_player_ids = pd.concat([onlymates['white_id'], onlymates['black_id']]).unique()

player_avg_ratings = {}



for player_id in all_player_ids:
    # Get ratings for the current player from both white and black ratings
    white_ratings = onlymates.loc[onlymates['white_id'] == player_id, 'white_rating']
    black_ratings = onlymates.loc[onlymates['black_id'] == player_id, 'black_rating']
    # Calculate the average rating for the player
    avg_rating = pd.concat([white_ratings, black_ratings]).mean()
    # Store the average rating for the player
    if (math.isnan(avg_rating)):
      avg_rating = 0
    player_avg_ratings[player_id] = avg_rating

sorted_player_avg_ratings = sorted(player_avg_ratings.items(), key=lambda x: x[1], reverse=True)

avg_rating_df = pd.DataFrame(sorted_player_avg_ratings, columns=['Player Name', 'Elo Rating'])

print(avg_rating_df)


              Player Name  Elo Rating
0               lance5500      2621.0
1               le_truand      2524.0
2              chess-stat      2500.0
3                  papasi      2489.0
4             amir2002zzz      2454.5
...                   ...         ...
6211             mata1234       808.0
6212        alliekatxoxo1       807.0
6213          epicchess66       796.0
6214     christinebitonti       795.0
6215  ragnarlothbrook_spb       784.0

[6216 rows x 2 columns]


In [10]:
avg_rating_df.loc[avg_rating_df['Player Name'] == "shahen07"]
sorted_player_skills_df.loc[sorted_player_skills_df['Player Name'] == "shahen07"]


,Player Name,Skill Mean,Skill SD


In [11]:
from scipy.stats import spearmanr

# Get actual and predicted rankings for all players
actual_rankings_all = []
predicted_rankings_all = []

for i, player_name in enumerate(sorted_player_skills_df['Player Name']):
    #print(player_name,avg_rating_df.loc[avg_rating_df['Player Name'] == player_name])
    actual_ranking = avg_rating_df.loc[avg_rating_df['Player Name'] == player_name].index[0]
    actual_rankings_all.append(actual_ranking)
    predicted_rankings_all.append(i)

# Compute the Spearman rank correlation coefficient for all players

spearman_corr_all, _ = spearmanr(predicted_rankings_all, actual_rankings_all)
print(f"Spearman Rank Correlation Coefficient for All Players: {spearman_corr_all}")

Spearman Rank Correlation Coefficient for All Players: 0.22309552926360113


In [12]:
print(len(predicted_rankings_all))
print(sorted_player_skills_df.loc[sorted_player_skills_df['Player Name'] == "yamaguchipolgar"])


6216
Empty DataFrame
Columns: [Player Name, Skill Mean, Skill SD]
Index: []


In [13]:
print(np.where(np.array(actual_rankings_all) == 252))

(array([3407]),)


In [14]:
print(avg_rating_df.loc[avg_rating_df['Player Name'] == "chesscarl"])

   Player Name  Elo Rating
70   chesscarl      2214.4


In [15]:
print(sorted_player_skills_df.loc[sorted_player_skills_df["Player Name"] == "lance5500"  ])
print(avg_rating_df.loc[avg_rating_df['Player Name'] == "lance5500"])
print("played games" , numgamess("lance5500"), "\n")

print(sorted_player_skills_df.loc[sorted_player_skills_df["Player Name"] == "fabian1104"  ])
print(avg_rating_df.loc[avg_rating_df['Player Name'] == "fabian1104"])
print("played games" , numgamess("fabian1104"), "\n")

print(sorted_player_skills_df.loc[sorted_player_skills_df["Player Name"] == "chess-stat"  ])
print(avg_rating_df.loc[avg_rating_df['Player Name'] == "chess-stat"])
print("played games" , numgamess("chess-stat"), "\n")



     Player Name  Skill Mean  Skill SD
6032   lance5500       3.649     1.637
  Player Name  Elo Rating
0   lance5500      2621.0
played games 44 

    Player Name  Skill Mean  Skill SD
727  fabian1104       4.528     1.545
    Player Name   Elo Rating
440  fabian1104  1949.615385
played games 40 

    Player Name  Skill Mean  Skill SD
737  chess-stat       1.322      2.22
  Player Name  Elo Rating
2  chess-stat      2500.0
played games 4 



In [ ]:
print(sorted_player_skills_df.loc[sorted_player_skills_df["Player Name"] == "brandonbos"  ])
print(avg_rating_df.loc[avg_rating_df['Player Name'] == "brandonbos"])
print("played games" , numgamess("brandonbos"), "\n")

print(sorted_player_skills_df.loc[sorted_player_skills_df["Player Name"] == "siindbad"  ])
print(avg_rating_df.loc[avg_rating_df['Player Name'] == "siindbad"])
print("played games" , numgamess("siindbad"), "\n")

print(sorted_player_skills_df.loc[sorted_player_skills_df["Player Name"] == "smilsydov"  ])
print(avg_rating_df.loc[avg_rating_df['Player Name'] == "smilsydov"])
print("played games" , numgamess("smilsydov"), "\n")